In [1]:
import ROOT
import stage0
import utils

%jsroot on

Welcome to JupyROOT 6.28/00


In [3]:
stage0.create_hist_models()


{'hTOF_mus': {'hTOF_mus_d0': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284585610>,
  'hTOF_mus_d1': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284584460>,
  'hTOF_mus_d2': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x2845842f0>,
  'hTOF_mus_d3': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284587880>,
  'hTOF_mus_d4': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x2845876a0>,
  'hTOF_mus_d5': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284585380>,
  'hTOF_mus_d6': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284577de0>,
  'hTOF_mus_d7': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284587070>,
  'hTOF_mus_d8': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284583b20>,
  'hTOF_mus_d9': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x284127b90>,
  'hTOF_mus_d10': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x2847b8810>,
  'hTOF_mus_d11': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x2847b8870>,
  'hTOF_mus_d12': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x2841283d0>,
  'hTOF_mus_d13': <cppyy.gbl.ROOT.RDF.TH1DModel object at 0x28

## Create TChain

Chain together multiple run files

In [ ]:
chain = ROOT.TChain("rawTree")

chain.Add("../data/rawroot_run_0014*")
# chain.Add("../data/raw0088_000.root")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_0015*")


# Apr 23, Ge, NaI, ALL RUNS BELOW 

# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_0064*")

# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_001[4-9]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_002[0-1]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_004[8-9]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_005[0-4]*")

# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_005[7-9]*") # detector position changes
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_006[0-3]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_006[7-9]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_007[0-3]*")
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_007[5-8]*")
   
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_007[9-9]*") # det 11 calibration changes here
# chain.Add("/Volumes/WD_BLACK/2023Apr/Ge/rawroot/rawroot_run_008[0-9]*")

# chain

1

In [ ]:
ROOT.EnableImplicitMT() # Enable multi-threading

df = ROOT.RDataFrame(chain)

df = stage0.filter_active_channels(df)

df = stage0.define_columns(df)



In [ ]:
# hist_dict = stage0.read_rawroot_to_dict("../data/rawroot_run_0014*")
hist_dict = stage0.read_rawroot_to_dict(chain, filter_active=False)
# hist_dict

Dataframe from TChain rawTree in file ../data/rawroot_run_0014_000.root

Property                Value
--------                -----
Columns in total           11
Columns from defines        4
Event loops run             0
Processing slots            8

Column          Type            Origin
------          ----            ------
Coin            UInt_t          Dataset
Egam            double          Define
En              double          Define
Flags           UInt_t          Dataset
PulseHeight     UShort_t        Dataset
Timestamp       ULong64_t       Dataset
detector        Int_t           Dataset
nTrigger        ULong64_t       Dataset
tof             ULong64_t       Dataset
tof_mus         double          Define
tof_ns          double          Define

## Write Histograms
Write histograms from dictionary to a root file maintaing heirerachy. 

In [ ]:
# # TODO: Make this a function

# # det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,19,-1,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] # det number for index of position number runs 13~54
# det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,-1,19,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] #det number for index of position number runs 55~90

# det_pos_map = ROOT.TH2I("det_pos_map","det_pos_map;position # ;detector # ",32,0,32,32,0,32)
# det_pos_graph = ROOT.TGraph()
# det_pos_graph.SetTitle("det_pos_graph")
# det_pos_graph.SetName("det_pos_graph")
# det_pos_graph.GetXaxis().SetTitle("positon #")
# det_pos_graph.GetYaxis().SetTitle("detector #")

# for ch in range(stage0.config['general']['numch']):
#     det_pos_map.SetBinContent(ch+1,det_number[ch]+1,1)
#     det_pos_graph.AddPoint(ch,det_number[ch])

# hist_dict[det_pos_map.GetName()] = det_pos_map
# hist_dict[det_pos_graph.GetName()] = det_pos_graph


In [ ]:
utils.write_dict_to_root(hist_dict, "../output/stage0_output.root")